# A binary classification Neural Network #

Purpose of this notebook :

- create a binary classification neural network from scratch
- practice oop programming
- practice docstring and commentaries

In [5]:
# Add useful librairies

import numpy as np
import matplotlib.pyplot as plt
import random

In [6]:
# First the full neural network

# The prupose is to predict from data
# let's create some data a XOR problematic

# Let's fix ou random seed
random.seed(42)

# A dataset of m example with 2 inputs shape(2 m)
X = np.random.randn(2, 100)
# Transorm inputs in O. or 1.
X = (X > 0.5) * 1.

# The labels (when x = (0 0) or (1 1) > False, x = (0 1) or (1 0) > True)
Y = (np.sum(X, axis=0, keepdims=True) == 1) * 1.

print('X : ', X[:,:5])
print('Y : ', Y[:,:5])

X :  [[1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1.]]
Y :  [[1. 1. 0. 1. 1.]]


In [7]:
# Initialisation
# List of hidden layers
hidden_layers = [4, 2]

# List of layers for binary classification
layers = [X.shape[0]] + hidden_layers + [1]

# Initialisation of list Weights and bias shape (output_layer(-1) layer) (output_layer(-1) 1)
W = [np.random.randn(layers[k], layers[k-1]) for k in range(1, len(layers))]
b = [np.zeros((layers[k], 1)) for k in range(1, len(layers))]

print('layers : ', layers)
print('W :')
for i in W:
    print(i.shape)
print('b :')
for i in b:
    print(i.shape)


layers :  [2, 4, 2, 1]
W :
(4, 2)
(2, 4)
(1, 2)
b :
(4, 1)
(2, 1)
(1, 1)


In [20]:
# Forward

# Let's define the activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Let's make a list of activation starting with the inputs matrix
activation = [X]

# Loop to pass through all neurons
for l in range(len(layers)-1):
    pre_activation = np.dot(W[l], activation[l]) + b[l]
    activation.append(sigmoid(pre_activation))

# check the activations shape    
    print(activation[l].shape)
print(activation[-1].shape)
    

(2, 100)
(4, 100)
(2, 100)
(1, 100)


In [21]:
# What about the loss and cost

prediction_inter = activation[-1]

# loss function
loss = -(Y * np.log(prediction_inter) + (1 - Y) * np.log(1 - prediction_inter))

# number of examples
m = X.shape[1]
cost = np.sum(loss) / m

print(loss.shape)
print(cost)

(1, 100)
0.6876592944705928


In [22]:
# Backward

# First the last layer
d_loss = -Y/activation[-1] + (1-Y)/(1-activation[-1])
d_activation = activation[-1] * (1 - activation[-1])
d_pre_activation = d_loss * d_activation

d_W = [np.dot(d_pre_activation, activation[-2].T)]
d_b = [np.sum(d_pre_activation, axis=1, keepdims=True) / m]

# Loop for the next layers
for l in reversed(range(1, len(layers)-1)):
    d_pre_activation = np.dot(W[l].T, d_pre_activation)
    
    d_W.insert(0, np.dot(d_pre_activation, activation[l-1].T))
    d_b.insert(0, np.sum(d_pre_activation, axis=1, keepdims=True) / m)

for i in d_W:
    print(i.shape)

for i in d_b:
    print(i.shape)
    

(4, 2)
(2, 4)
(1, 2)
(4, 1)
(2, 1)
(1, 1)


In [19]:
# updating the parameters

# Setting the learning rate
learning_rate = 0.5

# Loop for updating all weights and bias in the network
for i in range(len(layers)-1):
    W[i] -= learning_rate * d_W[i]
    b[i] -= learning_rate * d_b[i]
    
    print('new W', i, ' : ', W[i].shape)
    print('new b', i, ' : ', b[i].shape)
    

new W 0  :  (4, 2)
new b 0  :  (4, 1)
new W 1  :  (2, 4)
new b 1  :  (2, 1)
new W 2  :  (1, 2)
new b 2  :  (1, 1)
